In [2]:
dataframe_path = "default_path"
idx_layer_used = -1
list_comparison_levels = None
report_folder = None

In [ ]:
import re
from IPython.display import display, Markdown
import pandas as pd
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import pathlib
import seaborn as sns
import numpy as np

In [ ]:
report_folder = (
    pathlib.Path(dataframe_path).parent / "report"
    if report_folder is None
    else pathlib.Path(report_folder)
)
report_folder.mkdir(exist_ok=True, parents=True)
df = pd.read_csv(dataframe_path)
pattern = re.compile(r"^\d+: .*$")
layers_names = [col for col in df.columns if pattern.match(col)]

if list_comparison_levels is not None:
    df = df[df["ComparisonLevel"].isin(list_comparison_levels)]

mean_distances = df.groupby("ComparisonLevel").mean().filter(regex="(Conv|Linear)")

mean_distances_std = df.groupby("ComparisonLevel").std().filter(regex="(Conv|Linear)")

display(Markdown(f"# Analysis for dataset ***{dataframe_path}***"))

In [ ]:
# You might want to skip the analysis of the very initial layer as their value is massive and out of range and, finally, uninteresting as too close to pixel level. Set `start_from` to something from [0, len(layers_names) - 1]
start_from = 0

plt.rcParams["svg.fonttype"] = "none"

sns.set_style("white")


conditions = [
    np.array(mean_distances.iloc[i])[start_from:] for i in range(len(mean_distances))
]
conditions_std = [
    np.array(mean_distances_std.iloc[i])[start_from:]
    for i in range(len(mean_distances))
]
layer_index_name = mean_distances.columns[start_from:]

x = range(len(conditions[0]))
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
plt.axhline(y=0, linestyle=":", color="k", alpha=0.6, linewidth=1)

for idx, c in enumerate(conditions):
    plt.plot(x, c, label=mean_distances.index[idx], lw=2)

ax.set_xticks([])
plt.xticks(fontsize=10)
plt.yticks(fontsize=12)

ax.patch.set_edgecolor("black")
ax.patch.set_linewidth(1)
sns.despine()
plt.legend(prop={"size": 12}, edgecolor=(0, 0, 0, 1))
plt.ylabel("Distance Metric", fontsize=12)
plt.xlabel("Network Depth", fontsize=12)
network_name = "Network Name"
ax.annotate(
    network_name,
    color="k",
    xy=(0.2, 0.95),
    xycoords="axes fraction",
    size=15,
    fontweight="bold",
    bbox=dict(boxstyle="round", alpha=1, fc="wheat", ec="k"),
)
ax.tick_params(
    axis="y",
    left=True,
    labelbottom=True,
)
ax.tick_params(
    axis="x",
    bottom=True,
    labelbottom=True,
)
ax.set_xticks(x[::25], layer_index_name[::25], rotation=0)

plt.tight_layout()
[
    plt.savefig(
        str(pathlib.Path(report_folder) / f"lineplot_comparion_level_all_layers.{ext}"),
        facecolor="white",
    )
    for ext in ["png", "svg"]
]

display(
    Markdown(
        f"# All layers from ***{start_from}: (named {mean_distances.columns[start_from]})*** to the last one"
    )
)

In [ ]:
def box_plot_bunch_at_position(
    sample, x, ax, ll=None, cc=None, width=None, span=None, space=None, showmeans=True
):
    span = 0.35 if span is None else span
    space = 0.03 if space is None else space
    mm = len(sample) if len(sample) > 1 else 2
    width = span / (mm - 1) if width is None else width
    i = np.arange(0, len(sample))
    ll = [None] * len(sample) if ll is None else ll
    for iidx, d in enumerate(sample):
        bp = ax.boxplot(
            d,
            patch_artist=True,
            showfliers=False,
            positions=[x - span / 2 + width * i[iidx]],
            widths=width - space,
            labels=[ll[iidx]],
            boxprops={
                "fill": (0, 0, 0, 1),
                "lw": 0.5,
                "facecolor": f"C{iidx}" if cc is None else cc,
                "alpha": 1,
            },
            showmeans=showmeans,
            meanprops={"markerfacecolor": "r", "markeredgecolor": "k"},
            medianprops={"color": "k", "linestyle": "-", "linewidth": 0.5, "zorder": 2},
        )

In [ ]:
# Change here the layer depth you want to plot
lname = layers_names[-2]

fig, ax = plt.subplots(1, 1, figsize=(2, 2))
ax.tick_params(
    axis="y",
    left=True,
    labelbottom=True,
)

cl = df["ComparisonLevel"].unique()
dd = np.array([df[(df["ComparisonLevel"] == i)][lname] for i in cl])
[box_plot_bunch_at_position([d], 0, ax, cc="C0", width=None) for d in dd]
ax.set_xticklabels(cl)
plt.ylabel("Distance Metric")
[
    plt.savefig(
        str(pathlib.Path(report_folder) / f"barplot_aggregated.{ext}"),
        facecolor="white",
    )
    for ext in ["png", "svg"]
]
display(
    Markdown(f"# Distance metric for the Comparison Conditions at layer **{lname}**")
)

In [ ]:
# Change here the layer depth you want to plot
ll = layers_names[-2]


mm = sorted(df["MatchingLevels"].unique(), key=lambda x: int(x.strip("[]")))
cl = df["ComparisonLevel"].unique()

fig, ax = plt.subplots(1, 1, figsize=(6, 3))

for idx, m in enumerate(mm):
    dd = np.array(
        [
            df[(df["ComparisonLevel"] == cc) & (df["MatchingLevels"] == m)][ll]
            for cc in cl
        ]
    )

    box_plot_bunch_at_position(dd, idx, ax, width=0.2)

ax.tick_params(
    axis="y",
    left=True,
    labelbottom=True,
)
ax.tick_params(
    axis="x",
    bottom=True,
    labelbottom=True,
)
ax.set_xticks(np.arange(0, len(mm)))
ax.set_xticklabels([m.strip("[]") for m in mm])
custom_lines = [
    Line2D([0], [0], linestyle="", marker="s", color=f"C{idx}")
    for idx in range(0, len(cl))
]
ax.set_xlabel("Matching Levels")
ax.set_ylabel("Distance Metric")
ax.legend(custom_lines, cl, prop={"size": 12}, ncol=4, edgecolor="k", loc="upper left")


[
    plt.savefig(
        str(pathlib.Path(report_folder) / f"barplot_all_matching_levels.{ext}"),
        facecolor="white",
    )
    for ext in ["png", "svg"]
]

display(
    Markdown(
        f"# Distance metric for the Comparison Conditions and each Matching Levels at layer **{lname}**"
    )
)

In [ ]:
from statsmodels.stats.anova import AnovaRM

name_layer_used = layers_names[-2]
try:
    r = AnovaRM(
        data=df,
        depvar=name_layer_used,
        subject="MatchingLevels",
        within=["ComparisonLevel"],
        aggregate_func="mean",
    ).fit()

    display(
        Markdown(
            f"# Statistical Analysis with ANOVA Repeated Measures for Layer **{lname}**"
        )
    )
    print(r.anova_table)
except Exception as e:
    print(e)